In [2]:
import os 
import getopt
import sys

import random
import re

import numpy as np
import h5py
import pickle
import random
import copy
import pandas as pd
import math 

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Input, Masking, Lambda, concatenate, Bidirectional, Dense, Dropout, Flatten, Conv1D,BatchNormalization,  MaxPooling1D, Bidirectional, GRU, TimeDistributed
import tensorflow as tf
from tensorflow import keras



np.random.seed(1337) # for reproducibility
vocab = ["A", "G", "C", "T"]
indices = tf.range(len(vocab), dtype = tf.int64)
table_init = tf.lookup.KeyValueTensorInitializer(vocab,indices)
table = tf.lookup.StaticVocabularyTable(table_init, 1)
record_defaults = [
    tf.constant([''], dtype=tf.string),
    tf.constant([''], dtype=tf.string),
    tf.constant([''], dtype=tf.string),
    tf.constant([''], dtype=tf.string),  
    tf.constant([''], dtype=tf.string),
    tf.constant([''], dtype=tf.string),
    tf.constant([''], dtype=tf.string),  
    tf.constant([''], dtype=tf.string),
    tf.constant([''], dtype=tf.string),
]

# Nadav dataset

def data_reader(file, batch_size=100, n_parse_threads=4):
    """Method for reading the data in an optimized way, can be used inside model.fit()
    
    Args:
        file (_type_): path to csv file
        batch_size (int, optional): _description_. Defaults to 100.
        n_parse_threads (int, optional): _description_. Defaults to 4.

    Returns:
        dataset.batch: batch dataset object 
    """
    dataset = tf.data.TextLineDataset(file).skip(1)
    dataset = dataset.map(preprocess, num_parallel_calls=n_parse_threads)
    return dataset.batch(batch_size).prefetch(1)

def preprocess(record):
    """Preprocessing method of a dataset object, one-hot-encodes the data

    Args:
        record (_type_): _description_

    Returns:
        X (2D np.array): one-hot-encoded input sequence
        Y (1D np.array): MPRA measurements for each cell state
    """
    # Get fields from the data
    fields = tf.io.decode_csv(record, record_defaults=record_defaults)
    
    # One-hot-encode data
    chars = tf.strings.bytes_split(fields[0])
    chars_indices = table.lookup(chars)
    
    # Create a mask for out-of-vocabulary characters
    oov_mask = tf.equal(chars_indices, len(vocab))
    
    # Explicitly cast the value to int64
    chars_indices = tf.where(oov_mask, tf.constant(len(vocab), dtype=tf.int64), chars_indices)
    
    X = tf.one_hot(chars_indices, depth = len(vocab))
    
    # Combine y for each cell type into one vector 
    Y = tf.stack(fields[2:])
    
    # Replace missing values with -1
    Y= tf.where(tf.equal(Y,  "nan"), ["-1"], Y)
    Y= tf.where(tf.equal(Y,  "NA"), ["-1"], Y)
    Y = tf.strings.to_number(Y, tf.float32)
    return X,Y

# Get first item of the dataset to get the shape of the input data
for element in data_reader("/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/no_upper_LibA_wide_pivot_all_states.csv"):
    input_shape = element[0].shape
    output_shape = element[1].shape
    
print(output_shape)
print(input_shape)

(92, 7)
(92, 262, 4)


In [3]:
# Define the masking layer with mask_value=0
masking_layer = tf.keras.layers.Masking(mask_value=0.0)

# Example input tensor with shape (batch_size, sequence_length, vocab_size)
input_tensor = tf.constant([
    [[0, 0, 0, 0, 0], [1, 0, 0, 0, 0], [0, 0, 0, 1, 0]],
    [[0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]]
], dtype=tf.float32)

# Apply the masking layer to the input tensor
masked_tensor = masking_layer(input_tensor)

print(masked_tensor)
masking_layer.compute_mask(input_tensor)

tf.Tensor(
[[[0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0.]]

 [[0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0.]]], shape=(2, 3, 5), dtype=float32)


<tf.Tensor: shape=(2, 3), dtype=bool, numpy=
array([[False,  True,  True],
       [ True,  True, False]])>

In [4]:
# CROSS VALIDATION (10 fold)
from sklearn.model_selection import train_test_split, KFold

whole_data = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/no_upper_LibA_wide_pivot_all_states.csv"
out_data = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/"
# Split the data in two partitions
whole_data = pd.read_csv(whole_data)
k = 10
kf = KFold(n_splits = k, shuffle = True, random_state = 2008)

o=1
# For each fold we split again to get the third partition
for i in kf.split(whole_data):
    
    # Extract data from fold
    train = whole_data.iloc[i[0]]
    test =  whole_data.iloc[i[1]]
    
    # Split into train, validation and test
    valid, validation = train_test_split(train, test_size=0.111, random_state=42)
    
    # Randomize motifs
    #train = randomize_motifs_and_augment(train, num_augmentations=100)
    #train["rnd_seq"] = train['rnd_seq'].str.lower() 
    #test = randomize_motifs_and_augment(test, num_augmentations=100)
    #test["rnd_seq"] = test['rnd_seq'].str.lower() 
    #validation = randomize_motifs_and_augment(validation, num_augmentations=100)
    #validation["rnd_seq"] = validation['rnd_seq'].str.lower() 
    
    train.astype(str).to_csv(out_data+"all_states_no_upper_CV"+str(o)+"_LibA_wide_pivot_state3_train.csv", index=False)
    test.astype(str).to_csv(out_data+"all_states_no_upper_CV"+str(o)+"_LibA_wide_pivot_state3_test.csv", index=False)
    validation.astype(str).to_csv(out_data+"all_states_no_upper_CV"+str(o)+"_LibA_wide_pivot_state3_validation.csv", index=False)
    o+=1
    

In [5]:
# Here we initialize the df where each fold test prediction will be appended to
# the list containing the correlations of each fold is also initialized
corr_list = []

# We define a custom normalization layer to then compile on the model
class CustomNormalization(Layer):
    """Custom normalization layer that normalizes the output of the neural network"""
    def __init__(self, **kwargs):
        super(CustomNormalization, self).__init__(**kwargs)
        
    def build(self, input_shape):
        # Add trainable variables for mean and standard deviation
        self.mean = self.add_weight("mean", shape=(1,), initializer="zeros", trainable=True)
        self.stddev = self.add_weight("stddev", shape=(1,), initializer="ones", trainable=True)
        super(CustomNormalization, self).build(input_shape)  # Be sure to call this at the end

    def call(self, inputs):
        # Normalize the inputs using the learned mean and standard deviation
        return (inputs - self.mean) / (self.stddev + 1e-8)

# We define the method to compute the pearson correlation between prediction and ground truth in the multi_head case
def pearson_correlation_multi_head(predictions, ground_truth, mask_value=-1):
    """Computes Pearson Correlation between predictions and ground truth for each column
    Args:
        predictions (np.array): 2D array of prediction values (N, 7)
        ground_truth (np.array): 2D array of ground truth values (N, 7)
        mask_value (float): Value in ground truth to be ignored in correlation computation

    Returns:
        correlations (np.array): 1D array of Pearson correlations for each column
    """
    # Ensure predictions and ground_truth have the same shape
    assert predictions.shape == ground_truth.shape, "Input shapes do not match"

    n_columns = predictions.shape[1]
    correlations = np.zeros(n_columns)

    for col in range(n_columns):
        x = predictions[:, col]
        y = ground_truth[:, col]

        # Exclude values in ground truth equal to mask_value
        valid_indices = (y != mask_value)
        x = x[valid_indices]
        y = y[valid_indices]

        if len(x) == 0 or len(y) == 0:
            # If no valid values, set correlation to NaN
            correlations[col] = np.nan
        else:
            # Calculate mean of x and y
            mean_x = np.mean(x)
            mean_y = np.mean(y)

            # Calculate the numerator and denominators of the correlation coefficient
            numerator = np.sum((x - mean_x) * (y - mean_y))
            denominator_x = np.sqrt(np.sum((x - mean_x) ** 2))
            denominator_y = np.sqrt(np.sum((y - mean_y) ** 2))

            # Calculate the correlation coefficient
            correlation = numerator / (denominator_x * denominator_y)
            correlations[col] = correlation

    return correlations



import matplotlib.pyplot as plt
%matplotlib inline
# Define plotting function of loss
def create_plots(history):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    plt.clf()
    

In [6]:
# Define a custom loss function
class MaskedMSE(tf.keras.losses.Loss):
    """Computes the MSE loss and prevents missing values backpropagation (previously replaced by -1.0)

    Args:
        tf (_type_): _description_
    """
    def __init__(self, mask_value=-1, **kwargs):
        super(MaskedMSE, self).__init__(**kwargs)
        self.mask_value = mask_value

    def call(self, y_true, y_pred):
        # Create a mask for valid elements (not equal to the specified mask_value)
        mask = tf.math.not_equal(y_true, self.mask_value)

        # Compute MSE loss only for valid elements
        loss = tf.where(mask, tf.square(y_true - y_pred), 0.0)

        # Calculate the mean loss
        mean_loss = tf.reduce_sum(loss) / tf.reduce_sum(tf.cast(mask, dtype=tf.float32))

        return mean_loss

In [7]:
from tensorflow.keras.optimizers import Adam

corr_coefficients = {}

df_test_10folds  = pd.DataFrame(columns=['State_3E',
                                         "seq",
                                         "avg_prediction_State_1M",
                                         "avg_prediction_State_2D",
                                         "avg_prediction_State_3E",
                                         "avg_prediction_State_4M",
                                         "avg_prediction_State_5M",
                                         "avg_prediction_State_6N",
                                         "avg_prediction_State_7M"])
# We iterate through each of the train folds to train, test and validate the model

for i in range(1,11):
    #Define inputs
    input_path_train = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/all_states_no_upper_CV"+str(i)+"_LibA_wide_pivot_state3_train.csv"
    input_path_valid = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/all_states_no_upper_CV"+str(i)+"_LibA_wide_pivot_state3_validation.csv"
    input_path_test = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/all_states_no_upper_CV"+str(i)+"_LibA_wide_pivot_state3_test.csv"
    
    # Read test data to then predict
    df_test = pd.read_csv(input_path_test)
    df_test["fold"] = str(i)
    
    predictions_sum = np.zeros((df_test.shape[0], 7))
    
    # Get first item of the dataset to get the shape of the input data
    for element in data_reader(input_path_train):
        input_shape = element[0].shape
    
    correlation_ensemble = []
    # Initialize vector containing predictions
    
    for i in range(1,11):
        
        tf.random.set_seed(i)
        
        # Define and compile model
        inputs = Input(shape=(input_shape[1],input_shape[2]), name="inputs")
        inputs = Masking()(inputs)
        layer = Conv1D(250, kernel_size=7, strides=1, activation='relu', name="conv1")(inputs)  # 250 7 relu
        layer = Dropout(0.5)(layer)
        layer = BatchNormalization()(layer)
        layer = Conv1D(250, 8, strides=1, activation='softmax', name="conv2")(layer)  # 250 8 softmax
        layer = BatchNormalization()(layer)
        layer = MaxPooling1D(pool_size=2, strides=None, name="maxpool1")(layer)
        layer = Dropout(0.3)(layer)
        layer = Conv1D(250, 3, strides=1, activation='softmax', name="conv3")(layer)  # 250 3 softmax
        layer = BatchNormalization()(layer)
        layer = Dropout(0.5)(layer)
        layer = Conv1D(100, 2, strides=1, activation='softmax', name="conv4")(layer)  # 100 3 softmax
        layer = BatchNormalization()(layer)
        layer = MaxPooling1D(pool_size=1, strides=None, name="maxpool2")(layer)
        layer = Dropout(0.5)(layer)
        layer = Flatten()(layer)
        layer = Dense(300, activation='sigmoid')(layer)  # 300
        layer = Dropout(0.5)(layer)
        layer = Dense(200, activation='sigmoid')(layer)  # 300
        predictions = Dense(7, activation='linear')(layer)
        norm_predictions = CustomNormalization()(predictions)  # Assuming "predictions" is your existing output

        model = Model(inputs=inputs, outputs=norm_predictions)
        model.summary()

        # compile model
        model.compile(optimizer= Adam(clipvalue=1.0),  # You can adjust the clipvalue as needed
                    loss=MaskedMSE(mask_value=-1),
                    metrics=["mse"]
                    )

        # Run model
        history=model.fit(data_reader(input_path_train, batch_size=100),
                                epochs=20,
                                validation_data=data_reader(input_path_valid,batch_size=500),
                                callbacks=None,
                                verbose=1)    

        # We predict the test data
        predicted = model.predict(data_reader(input_path_test, batch_size=500))

        # Sum the predicted values (will be averaged at the end of the 10 iterations)
        predictions_sum =  np.add(predictions_sum, predicted)
        
        
        # We reed the data in the same order to compute the correlation score
        test_data = data_reader(input_path_test,batch_size=500)
        test_tensor = np.empty(shape=[0,7])
        for batch in test_data:
            test_tensor = np.append(test_tensor, batch[1], axis=0)
            
        # Append correlation coefficient and append to previous
        corr_coefficient = pearson_correlation_multi_head(test_tensor, predicted, mask_value=-1)
        correlation_ensemble.append(corr_coefficient)
        avg_corr_coefficient = 1
        
        
    # We fill the dataframe with predictions and fold annotation
    
    predicted = predictions_sum/10 # We divide by the number of iterations
    
    df_test["avg_prediction_State_1M"] = predicted[:,0]
    df_test["avg_prediction_State_2D"] = predicted[:,1]
    df_test["avg_prediction_State_3E"] = predicted[:,2]
    df_test["avg_prediction_State_4M"] = predicted[:,3]
    df_test["avg_prediction_State_5M"] = predicted[:,4]
    df_test["avg_prediction_State_6N"] = predicted[:,5]
    df_test["avg_prediction_State_7M"] = predicted[:,6]
    df_test["fold"] = str(i)
    df_test["partition"] = "test"
    corr_coefficients["fold"+str(i)] = avg_corr_coefficient

    
    # Append fold to previous folds
    df_test_10folds = pd.concat([df_test_10folds, df_test], ignore_index=True)    
    

# Save the results for all folds
df_test_10folds.to_csv("/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/ensemble_multihead_motifs_LibA_wide_pivot_state3_test_predicted_cv10fold.csv", index=False)

model_path = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/motifs_test_multihead_last_cv.h5"
model.save_weights(model_path, save_format='h5') 

df_test_10folds

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 262, 4)]          0         
                                                                 
 conv1 (Conv1D)              (None, 256, 250)          7250      
                                                                 
 dropout (Dropout)           (None, 256, 250)          0         
                                                                 
 batch_normalization (Batch  (None, 256, 250)          1000      
 Normalization)                                                  
                                                                 
 conv2 (Conv1D)              (None, 249, 250)          500250    
                                                                 
 batch_normalization_1 (Bat  (None, 249, 250)          1000      
 chNormalization)                                            

2023-11-28 11:00:14.679411: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-11-28 11:00:14.693720: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2023-11-28 11:00:14.804540: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-11-28 11:00:17.098172: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f94a9cd6410 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-28 11:00:17.098211: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2023-11-28 11:00:17.102527: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-11-28 11:00:17.190879: I ./tensorflow/compiler/jit/device_comp

77/77 [==============================] - 10s 42ms/step - loss: 0.0686 - mse: 0.3583 - val_loss: 0.0608 - val_mse: 0.3969
Epoch 2/20
 4/77 [>.............................] - ETA: 1s - loss: 0.0284 - mse: 0.3402

2023-11-28 11:00:22.811797: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 15338174185544872159
2023-11-28 11:00:22.811834: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 16128581703658883462
2023-11-28 11:00:22.811847: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 6820603868339063414


77/77 [==============================] - 2s 21ms/step - loss: 0.0226 - mse: 0.3137 - val_loss: 0.0602 - val_mse: 0.3936
Epoch 3/20
77/77 [==============================] - 2s 22ms/step - loss: 0.0181 - mse: 0.3085 - val_loss: 0.0518 - val_mse: 0.3691
Epoch 4/20
77/77 [==============================] - 2s 23ms/step - loss: 0.0158 - mse: 0.3062 - val_loss: 0.0410 - val_mse: 0.3498
Epoch 5/20
77/77 [==============================] - 2s 23ms/step - loss: 0.0149 - mse: 0.3048 - val_loss: 0.0341 - val_mse: 0.3247
Epoch 6/20
77/77 [==============================] - 2s 23ms/step - loss: 0.0139 - mse: 0.3039 - val_loss: 0.0281 - val_mse: 0.3163
Epoch 7/20
77/77 [==============================] - 2s 23ms/step - loss: 0.0135 - mse: 0.3034 - val_loss: 0.0239 - val_mse: 0.2999
Epoch 8/20
77/77 [==============================] - 2s 22ms/step - loss: 0.0132 - mse: 0.3030 - val_loss: 0.0176 - val_mse: 0.2864
Epoch 9/20
77/77 [==============================] - 2s 22ms/step - loss: 0.0129 - mse: 0.3024 

...

77/77 [==============================] - 2s 21ms/step - loss: 0.0217 - mse: 0.3076 - val_loss: 0.0355 - val_mse: 0.3438
Epoch 3/20
77/77 [==============================] - 2s 21ms/step - loss: 0.0175 - mse: 0.3035 - val_loss: 0.0315 - val_mse: 0.3330
Epoch 4/20
77/77 [==============================] - 2s 21ms/step - loss: 0.0157 - mse: 0.3025 - val_loss: 0.0300 - val_mse: 0.3147
Epoch 5/20
77/77 [==============================] - 2s 20ms/step - loss: 0.0146 - mse: 0.3000 - val_loss: 0.0301 - val_mse: 0.3033
Epoch 6/20
77/77 [==============================] - 2s 20ms/step - loss: 0.0145 - mse: 0.3009 - val_loss: 0.0298 - val_mse: 0.2973
Epoch 7/20
77/77 [==============================] - 2s 21ms/step - loss: 0.0141 - mse: 0.3005 - val_loss: 0.0283 - val_mse: 0.2920
Epoch 8/20
77/77 [==============================] - 2s 21ms/step - loss: 0.0139 - mse: 0.2995 - val_loss: 0.0223 - val_mse: 0.2961
Epoch 9/20
77/77 [==============================] - 2s 21ms/step - loss: 0.0136 - mse: 0.3001 

2023-11-28 12:04:19.797772: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 15338174185544872159
2023-11-28 12:04:19.797865: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 16128581703658883462
2023-11-28 12:04:19.797888: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 6820603868339063414


77/77 [==============================] - 2s 22ms/step - loss: 0.0193 - mse: 0.3066 - val_loss: 0.0371 - val_mse: 0.3267
Epoch 3/20
77/77 [==============================] - 2s 22ms/step - loss: 0.0163 - mse: 0.3028 - val_loss: 0.0313 - val_mse: 0.3214
Epoch 4/20
77/77 [==============================] - 2s 21ms/step - loss: 0.0152 - mse: 0.3020 - val_loss: 0.0313 - val_mse: 0.3104
Epoch 5/20
77/77 [==============================] - 2s 21ms/step - loss: 0.0146 - mse: 0.2999 - val_loss: 0.0297 - val_mse: 0.3207
Epoch 6/20
77/77 [==============================] - 2s 21ms/step - loss: 0.0142 - mse: 0.2997 - val_loss: 0.0295 - val_mse: 0.3056
Epoch 7/20
77/77 [==============================] - 2s 21ms/step - loss: 0.0138 - mse: 0.2997 - val_loss: 0.0273 - val_mse: 0.2940
Epoch 8/20
77/77 [==============================] - 2s 21ms/step - loss: 0.0138 - mse: 0.2998 - val_loss: 0.0169 - val_mse: 0.2928
Epoch 9/20
77/77 [==============================] - 2s 22ms/step - loss: 0.0134 - mse: 0.2992 

2023-11-28 12:05:31.452672: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 11978862840496074443


,State_3E,seq,avg_prediction_State_1M,avg_prediction_State_2D,avg_prediction_State_3E,avg_prediction_State_4M,avg_prediction_State_5M,avg_prediction_State_6N,avg_prediction_State_7M,CRS,State_1M,State_2D,State_4M,State_5M,State_6N,State_7M,fold,partition
0,-0.003656,aggaccggatcaactaaacaactcaaacaagggctaatataaccca...,0.119132,0.128859,0.092132,0.066288,0.089367,0.121304,0.093385,LibA.Seq7829,0.220825,0.091926,0.069263,NaN,-0.184409,NaN,10,test
1,0.163263,aggaccggatcaactaaacactagtcatacttaaaaattgcaagga...,0.032433,0.039961,0.010296,0.012464,0.026709,0.010235,0.045405,LibA.Seq271,0.113779,0.063990,0.075333,NaN,0.060112,NaN,10,test
2,-0.059611,aggaccggatcaactaaacaggttctgacgtatgctcctctatgga...,0.046036,0.052255,0.020450,0.015620,0.034823,0.025535,0.050039,LibA.Seq4548,-0.091269,0.008748,-0.032317,NaN,-0.123168,NaN,10,test
3,-0.060434,aggaccggatcaactaaacccgagcctgcctagccctagcttctct...,0.030421,0.042230,0.011959,0.011131,0.021949,0.012314,0.043639,LibA.Seq4582,0.011752,-0.038761,0.014523,0.051586,-0.036038,0.068024,10,test
4,0.255777,aggaccggatcaactaaacggagcagagttagtgtcaggtcaaaaa...,0.158272,0.160801,0.133515,0.092224,0.126785,0.167793,0.117303,LibA.Seq2863,0.269027,0.181167,0.099999,NaN,0.175921,NaN,10,test
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8487,-0.052776,aggaccggatcaacttttggtcggttgacggtcgccttgattattc...,0.045518,0.050633,0.030000,0.008594,0.037687,0.045324,0.039015,LibA.Seq4154,-0.012530,-0.020680,-0.033009,-0.013301,-0.118172,-0.027401,10,test
8488,0.254768,aggaccggatcaactttttcagtgaaagatcaccgcgggatctcac...,0.196505,0.194129,0.160741,0.114982,0.161372,0.236327,0.141398,LibA.Seq8531,0.320753,0.342002,0.172941,0.249537,0.382781,0.211203,10,test
8489,-0.067905,aggaccggatcaacttttttagtaaaactcttaaacagtgattaca...,0.124789,0.090422,0.091210,0.042598,0.092363,0.121449,0.103937,LibA.Seq6744,0.073526,0.130104,0.203765,NaN,NaN,NaN,10,test
8490,0.113337,aggaccggatcaacttttttatctggttatcattctagtctagtgc...,0.061113,0.063943,0.043401,0.017668,0.048604,0.060335,0.049960,LibA.Seq1298,0.090932,0.186463,0.107309,NaN,0.218181,0.209876,10,test


In [9]:
corr_coefficients

{'fold10': 1}

In [ ]:
df_test_10folds
